## Analise e Previsão de Preços de Imóveis em Brasília

O mercado imobiliário apresenta grande variabilidade de preços, influenciada por fatores como **localização**, **metragem** e **características do imóvel**. Em cidades como **Brasília**, essa variação se torna ainda mais relevante devido à heterogeneidade entre regiões administrativas e ao perfil destinto dos imóveis ofertados.  
Neste contexto, a análise de dados imobiliários pode auxiliar na compreensão do padrões de precificação e na construção de modelos capazes de fornecer estimativas de valor com base em características observáveis dos imóveis.  
#### Objetivo do Projeto
O objetivo desse projeto é desenvolver uma pipiline de dados completo, contemplando:
- limpeza e preparação de dados provenientes de anúncios imobiliários em Brasília;
- análise exploratório para identificação de padrões regionais e estruturais;
- desenvolvimento de um modelo de **machine learning** para estimativa de preços de imóveis;
- disponibilização do resultados por meio de uma aplicação web interativa, permitindo a exploração dos dados e a realização de previsões.  
O projeto tem caráter aplicado, como foco em reproduzir desafios reais de projetos de dados, incluindo inconsistências, valores ausente e limitações inerentes à coleta de dados **via web scraping**.  
#### Descrição do Dataset

O conjunto de dados utilizado neste projeto foi obtido a partir de anúncios de públicos de imóveis localizados em Brasília. As principais variáveis disponíveis incluem:  
- **Região**: região administrativa do imóvel;
- **Preço**: Valor anunciado do imóvel;
- **Área (m²)**: metragem do imóvel;
- **Quantidade de Quartos**: número de dormitórios.  
Como esperado em dados coletados a partir de fontes abertas, o dataset apresenta valores ausente e inconsistências, que serão tratados ao longo da etapa de limpeza e preparação dos dados.  
#### Estrutura do Notebook  
1. Carregamento e inspeção inicial dos dados;
2. Padronização de tipos e formatos;
3. Tratamento de valores ausentes;
4. Identificação e tratamento de inconsistências;
5. Geração de um dataset limpo para as etapas posteriores de análise e modelagem.  
#### Considerações Iniciais  
Ressalta-se que o modelo desenvolvido neste projeto tem como finalidade fornecer estimativas aproximadas, não devendo ser interpretado como avaliação oficial de mercado. Os resultados devem ser analisados dentro das limitações impostas pela natureza dos dados e pelas premissas adotadas ao longo do pipiline.

#### 1. Importação das Bibliotecas

Nesta etapa são importadas as bibliotecas necessárias para a manipulação, análise e visualização dos dados ao longo do projeto.  
Optou-se por utilizar bibliotecas amplamente adotadas no ecossistema de ciência de dados em Python, visando clareza, reprodutibilidade e boas práticas.


In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

#### 1.1 Carregamento do Dataset

O dataset utilizado neste projeto foi previamente coletado por meio de web scraping de anúncios imobiliários localizados em Brasília.  
Nesta etapa, os dados são carregados para inspeção inicial, permitindo a compreensão de sua estrutura, dimensões e tipos de variáveis disponíveis.


In [2]:
# Ajuste conforme o necessario

df_orig = pd.read_csv(r'C:\analise_imoveis\data\raw\imoveis_dfimoveis.csv')

# Sempre faço uma copia e trabalho com ela
df = df_orig.copy()

In [3]:
df.head()

,titulo,preco,area,quartos,link
0,"SQNW 105 Bloco A NOROESTE, BRASILIA",Sob Consulta,194 a 204 m²,4 quartos,https://www.dfimoveis.com.br/imovel/lancamento...
1,"SQN 402, ASA NORTE, BRASILIA",1.400.000,141 m²,2 Quartos,https://www.dfimoveis.com.br/imovel/apartament...
2,"SQSW 303 Bloco J, SUDOESTE, BRASILIA",1.529.000,79 m²,2 Quartos,https://www.dfimoveis.com.br/imovel/apartament...
3,"SQNW 307 Bloco A, NOROESTE, BRASILIA",1.395.000,85 m²,2 Quartos,https://www.dfimoveis.com.br/imovel/apartament...
4,"SQB 1 GUARA I, GUARA",Sob Consulta,109 m²,3 quartos,https://www.dfimoveis.com.br/imovel/lancamento...


In [4]:
# De cara, nao precimaos da coluna 'link'
df.drop(columns='link', inplace=True)

In [5]:
df.head()

,titulo,preco,area,quartos
0,"SQNW 105 Bloco A NOROESTE, BRASILIA",Sob Consulta,194 a 204 m²,4 quartos
1,"SQN 402, ASA NORTE, BRASILIA",1.400.000,141 m²,2 Quartos
2,"SQSW 303 Bloco J, SUDOESTE, BRASILIA",1.529.000,79 m²,2 Quartos
3,"SQNW 307 Bloco A, NOROESTE, BRASILIA",1.395.000,85 m²,2 Quartos
4,"SQB 1 GUARA I, GUARA",Sob Consulta,109 m²,3 quartos


#### 1.2 Visão Geral dos Dados

Antes de qualquer etapa de limpeza ou transformação, é fundamental compreender a estrutura geral do conjunto de dados.  
Nesta seção são analisados aspectos como:

- dimensões do dataset;
- tipos das variáveis;
- presença de valores ausentes;
- estatísticas descritivas iniciais.

Essa análise preliminar orientará as decisões tomadas nas etapas subsequentes de limpeza e preparação dos dados.


In [6]:
df.shape

(17732, 4)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17732 entries, 0 to 17731
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   titulo   17729 non-null  object
 1   preco    17729 non-null  object
 2   area     17724 non-null  object
 3   quartos  15121 non-null  object
dtypes: object(4)
memory usage: 554.3+ KB


In [8]:
# Quantidade de valores nulos
df.isna().sum().sort_values(ascending=False)

quartos    2611
area          8
preco         3
titulo        3
dtype: int64

In [9]:
df.describe(include=object)

,titulo,preco,area,quartos
count,17729,17729,17724,15121
unique,5609,1896,1425,64
top,"RODOVIA BR 0020 KM 12,5, ALTO DA BOA VISTA, SO...",850.000,400 m²,3 Quartos
freq,207,225,400,4821


#### 1.3 Considerações Iniciais sobre a Qualidade dos Dados

A inspeção inicial do dataset evidencia que todas as variáveis foram carregadas como do tipo `object`.  
Isso ocorre devido ao fato de os dados terem sido coletados a partir de anúncios imobiliários, nos quais informações numéricas frequentemente são acompanhadas de textos, unidades de medida e símbolos monetários.

Como consequência, não é possível aplicar diretamente métodos estatísticos descritivos sobre variáveis como preço, área e quantidade de quartos sem uma etapa prévia de padronização e conversão de tipos.

Além disso, observa-se a presença de valores ausentes em algumas colunas, com destaque para a variável **quartos**, que apresenta uma quantidade significativa de registros faltantes.  
Esse cenário reforça a necessidade de uma abordagem cuidadosa no tratamento dos dados, evitando decisões arbitrárias que possam introduzir vieses ao conjunto final.

Durante a inspeção inicial, identificou-se a existência de uma coluna contendo apenas o link do anúncio original.  
Considerando que essa informação não agrega valor às etapas de análise exploratória ou modelagem preditiva, optou-se por sua remoção do dataset.


#### 2. Introdução à Limpeza e Tratamento dos Dados

A etapa de limpeza e preparação dos dados tem como objetivo transformar o dataset bruto em uma base consistente e adequada para análise exploratória e modelagem preditiva.

As principais ações previstas nesta etapa incluem:

- padronização dos formatos das variáveis;
- remoção de símbolos, unidades e textos não numéricos;
- conversão de tipos de dados para formatos apropriados;
- tratamento de valores ausentes com base em critérios estatísticos;

Todas as decisões adotadas ao longo deste processo serão devidamente justificadas, considerando as limitações inerentes aos dados coletados.

A partir deste ponto, o foco será a padronização e conversão das variáveis principais, iniciando pelo tratamento das colunas de preço, área e quantidade de quartos, de modo a viabilizar análises estatísticas e o desenvolvimento de modelos de machine learning.


#### 2.1 Padronização dos Nomes das Colunas

Como primeira etapa prática da limpeza, os nomes das colunas foram padronizados para letras maiúsculas.  
Essa padronização facilita a leitura do código, reduz ambiguidades e contribui para a consistência ao longo do projeto.

In [10]:
df.columns = df.columns.str.upper()

df.columns

Index(['TITULO', 'PRECO', 'AREA', 'QUARTOS'], dtype='object')

#### 2.2 Extração da Região Administrativa (RA)

A coluna **TITULO** contém informações detalhadas de endereço, como quadra, bloco e complementos, que não são relevantes para os objetivos deste projeto.  
Dessa forma, optou-se por extrair apenas a **Região Administrativa (RA)** do Distrito Federal, reduzindo ruído e preservando informações essenciais para análise e modelagem.

A extração irá ser realizada com base em uma lista previamente definida de regiões administrativas oficiais, garantindo padronização e consistência dos dados.

In [11]:
import unicodedata
import re

In [12]:
# Reduz inconsistências causadas por variações ortográficas
def normalizar_texto(texto):
    if pd.isna(texto):
        return texto
    texto = unicodedata.normalize('NFKD', texto)
    texto = texto.encode('ASCII', 'ignore').decode('utf-8')
    return texto.upper()

In [13]:
df['endereco_norm'] = df['TITULO'].apply(normalizar_texto)

In [14]:
regioes_adm = [
    "PLANO PILOTO", "GAMA", "TAGUATINGA", "BRAZLANDIA", "SOBRADINHO",
    "PLANALTINA", "PARANOA", "NUCLEO BANDEIRANTE", "CEILANDIA",
    "GUARA I", "GUARA II", "GUARA",
    "CRUZEIRO", "SAMAMBAIA SUL", "SAMAMBAIA",
    "SANTA MARIA", "SAO SEBASTIAO", "RECANTO DAS EMAS",
    "LAGO SUL", "LAGO NORTE", "CANDANGOLANDIA",
    "AGUAS CLARAS", "RIACHO FUNDO II", "RIACHO FUNDO",
    "SUDOESTE", "OCTOGONAL", "VARJAO",
    "PARK WAY", "SCIA", "SOBRADINHO II",
    "JARDIM BOTANICO", "ITAPOA", "SIA",
    "VICENTE PIRES", "FERCAL",
    "SOL NASCENTE", "POR DO SOL", "ARNIQUEIRA",
    "TAQUARI", "NOROESTE", "ASA NORTE", "ASA SUL",
    "PARK SUL", "JARDINS MANGUEIRAL"
]


In [15]:
regioes_ordenadas = sorted(regioes_adm, key=len, reverse=True)

regex_ra = r'\b(' + '|'.join(map(re.escape, regioes_ordenadas)) + r')\b'

In [16]:
df['BAIRRO'] = df['endereco_norm'].str.extract(regex_ra, expand=False)

In [17]:
df.head()

,TITULO,PRECO,AREA,QUARTOS,endereco_norm,BAIRRO
0,"SQNW 105 Bloco A NOROESTE, BRASILIA",Sob Consulta,194 a 204 m²,4 quartos,"SQNW 105 BLOCO A NOROESTE, BRASILIA",NOROESTE
1,"SQN 402, ASA NORTE, BRASILIA",1.400.000,141 m²,2 Quartos,"SQN 402, ASA NORTE, BRASILIA",ASA NORTE
2,"SQSW 303 Bloco J, SUDOESTE, BRASILIA",1.529.000,79 m²,2 Quartos,"SQSW 303 BLOCO J, SUDOESTE, BRASILIA",SUDOESTE
3,"SQNW 307 Bloco A, NOROESTE, BRASILIA",1.395.000,85 m²,2 Quartos,"SQNW 307 BLOCO A, NOROESTE, BRASILIA",NOROESTE
4,"SQB 1 GUARA I, GUARA",Sob Consulta,109 m²,3 quartos,"SQB 1 GUARA I, GUARA",GUARA I


In [18]:
df['BAIRRO'].value_counts()

BAIRRO
AGUAS CLARAS          2808
JARDIM BOTANICO       1950
SOBRADINHO            1300
ASA NORTE             1067
TAGUATINGA             935
VICENTE PIRES          931
NOROESTE               746
ASA SUL                741
PARK WAY               670
LAGO SUL               664
GUARA II               607
ARNIQUEIRA             574
LAGO NORTE             540
SUDOESTE               457
SAMAMBAIA              442
SAMAMBAIA SUL          404
CEILANDIA              384
GAMA                   323
GUARA I                245
PARK SUL               236
RIACHO FUNDO           129
PARANOA                106
CRUZEIRO               101
NUCLEO BANDEIRANTE      96
RIACHO FUNDO II         93
PLANALTINA              93
JARDINS MANGUEIRAL      92
RECANTO DAS EMAS        69
TAQUARI                 56
SANTA MARIA             53
GUARA                   47
OCTOGONAL               44
SAO SEBASTIAO           40
CANDANGOLANDIA          37
SIA                     36
BRAZLANDIA              36
SCIA                 

In [19]:
# Diminuindo granulidades
df['BAIRRO'] = df['BAIRRO'].replace({
    'GUARA I': 'GUARA',
    'GUARA II': 'GUARA',
    'SAMAMBAIA SUL': 'SAMAMBAIA',
    'RIACHO FUNDO II': 'RIACHO FUNDO'

})

In [20]:
# Conferindo
df['BAIRRO'].value_counts()

BAIRRO
AGUAS CLARAS          2808
JARDIM BOTANICO       1950
SOBRADINHO            1300
ASA NORTE             1067
TAGUATINGA             935
VICENTE PIRES          931
GUARA                  899
SAMAMBAIA              846
NOROESTE               746
ASA SUL                741
PARK WAY               670
LAGO SUL               664
ARNIQUEIRA             574
LAGO NORTE             540
SUDOESTE               457
CEILANDIA              384
GAMA                   323
PARK SUL               236
RIACHO FUNDO           222
PARANOA                106
CRUZEIRO               101
NUCLEO BANDEIRANTE      96
PLANALTINA              93
JARDINS MANGUEIRAL      92
RECANTO DAS EMAS        69
TAQUARI                 56
SANTA MARIA             53
OCTOGONAL               44
SAO SEBASTIAO           40
CANDANGOLANDIA          37
SIA                     36
BRAZLANDIA              36
SCIA                    15
SOL NASCENTE            10
ITAPOA                   8
VARJAO                   7
FERCAL               

In [21]:
df.head()

,TITULO,PRECO,AREA,QUARTOS,endereco_norm,BAIRRO
0,"SQNW 105 Bloco A NOROESTE, BRASILIA",Sob Consulta,194 a 204 m²,4 quartos,"SQNW 105 BLOCO A NOROESTE, BRASILIA",NOROESTE
1,"SQN 402, ASA NORTE, BRASILIA",1.400.000,141 m²,2 Quartos,"SQN 402, ASA NORTE, BRASILIA",ASA NORTE
2,"SQSW 303 Bloco J, SUDOESTE, BRASILIA",1.529.000,79 m²,2 Quartos,"SQSW 303 BLOCO J, SUDOESTE, BRASILIA",SUDOESTE
3,"SQNW 307 Bloco A, NOROESTE, BRASILIA",1.395.000,85 m²,2 Quartos,"SQNW 307 BLOCO A, NOROESTE, BRASILIA",NOROESTE
4,"SQB 1 GUARA I, GUARA",Sob Consulta,109 m²,3 quartos,"SQB 1 GUARA I, GUARA",GUARA


In [22]:
df['BAIRRO'].isna().sum()

np.int64(531)

In [23]:
df.drop(['TITULO', 'endereco_norm'], axis=1, inplace=True)

- Sem dados faltantes

#### 2.3 Extração e consolidação da coluna de região (BAIRRO)

Com os textos padronizados e a lista de regiões definida, foi realizado o processo de extração da região administrativa diretamente do título do anúncio.

O procedimento consistiu em:

- Identificar automaticamente a região administrativa presente no texto;

- Criar a coluna BAIRRO contendo apenas essa informação;

- Consolidar variações de nomenclatura, como:

    - _Guará I e Guará_ II → *Guará*
    - _Samambaia Sul_ → *Samambaia*
    - _Riacho Fundo II_ → *Riacho Fundo*

Ao final dessa etapa, o dataset passou a conter uma coluna limpa e padronizada de região administrativa.

Essa estrutura torna o conjunto de dados mais adequado para análises exploratórias, visualizações e modelagens futuras.

#### 3. Problemas identificados na coluna PREÇO

Durante a inspeção inicil, foram identificados os seguintes padrões de inconsistência:
- valores monetários armazenados como texto;
- Presença de strings não numéricas, como:
    - '_Sob Consulta_';
    - '_A partir de X_', entre outras;
    - Separadores de milhar representados por ponto (`.`), o que requer tratamento antes da conversão para o tipo numérico

Esses fatores impedem o cálculo direto de estatisticas descritivas, com média e mediana, além de inviabilizar o uso da variável em etapas posteriores.

In [24]:
# Limpando os texto e extraindo so numeros
df['VALOR'] = (
    df['PRECO']
    .str.replace('A partir de', '', regex=False)
    .str.replace('.', '', regex=False)
    .str.replace(',', '', regex=False)
    .str.extract(r'(\d+\.?\d*)')[0]
    .astype(float)
)

In [25]:
mediana_preco = df['VALOR'].median()

In [26]:
df['VALOR'].fillna(mediana_preco, inplace=True)

In [27]:
df.head(50)

,PRECO,AREA,QUARTOS,BAIRRO,VALOR
0,Sob Consulta,194 a 204 m²,4 quartos,NOROESTE,950000.0
1,1.400.000,141 m²,2 Quartos,ASA NORTE,1400000.0
2,1.529.000,79 m²,2 Quartos,SUDOESTE,1529000.0
3,1.395.000,85 m²,2 Quartos,NOROESTE,1395000.0
4,Sob Consulta,109 m²,3 quartos,GUARA,950000.0
5,450.000,58 m²,2 Quartos,AGUAS CLARAS,450000.0
6,3.580.000,186 m²,4 Quartos,SUDOESTE,3580000.0
7,850.000,90 m²,3 Quartos,AGUAS CLARAS,850000.0
8,Sob Consulta,33 m²,1 quarto,SAMAMBAIA,950000.0
9,850.000,68 m²,2 Quartos,GUARA,850000.0


In [28]:
df.loc[df['PRECO'] == 'Sob Consulta', ['PRECO', 'AREA', 'QUARTOS', 'BAIRRO', 'VALOR']]

,PRECO,AREA,QUARTOS,BAIRRO,VALOR
0,Sob Consulta,194 a 204 m²,4 quartos,NOROESTE,950000.0
4,Sob Consulta,109 m²,3 quartos,GUARA,950000.0
8,Sob Consulta,33 m²,1 quarto,SAMAMBAIA,950000.0
12,Sob Consulta,64 a 220 m²,2 a 3 quartos,NOROESTE,950000.0
39,Sob Consulta,159 m²,4 quartos,SUDOESTE,950000.0
47,Sob Consulta,175 m²,4 quartos,SUDOESTE,950000.0
51,Sob Consulta,115 a 149 m²,3 a 4 quartos,GUARA,950000.0
55,Sob Consulta,109 m²,3 quartos,GUARA,950000.0
67,Sob Consulta,82 a 90 m²,2 quartos,SUDOESTE,950000.0
86,Sob Consulta,69 m²,2 quartos,NOROESTE,950000.0


In [29]:
df[df['PRECO'] == 'Sob Consulta'].shape[0]

44

- Decidi remover todas as linhas que cotenham _'A partir de'_ e _'Sob Consulta'_.

In [30]:
df = df[~df['PRECO'].str.match(r'^(Sob Consulta| A partir de)', case=False, na=False)]

In [31]:
# Conferindo
df[df['PRECO'].str.contains('Sob Consulta| A partir de', case=False, na=False)]

,PRECO,AREA,QUARTOS,BAIRRO,VALOR


In [32]:
# Comparando o data set antes e depois
antes = df_orig.shape[0]
depois = df.shape[0]

print(f'Linhas antes: {antes}')
print(f'Linhas depois {depois}')
print(f'Removidas: {antes - depois}')

Linhas antes: 17732
Linhas depois 17688
Removidas: 44


##### Verificação da exclusão de registros  
Após observar que o número de linhas excluídas correspondem exatamente à quantidade de registros classificados como _“Sob Consulta”_, optei por realizar uma verificação mais robusta. Essa etapa tem como objetivo confirmar a presença de qualquer valor textual na coluna PREÇO, uma vez que a intenção é garantir que a variável contenha exclusivamente valores numéricos antes das próximas etapas de análise e modelagem.

In [33]:
# Vereficando varições de escrita ainda existentes
df.loc[
    df['PRECO'].astype(str).str.contains('sob|partir', case=False, na=False),
    'PRECO'
]

16       A partir de 881.826
20       A partir de 239.000
24       A partir de 637.750
28       A partir de 353.490
59       A partir de 959.482
63       A partir de 637.750
78     A partir de 1.645.886
82     A partir de 4.510.049
106    A partir de 2.345.127
133    A partir de 1.207.611
137      A partir de 599.000
141    A partir de 2.682.693
155    A partir de 1.886.956
167      A partir de 678.080
171    A partir de 1.979.473
175    A partir de 3.061.662
196      A partir de 417.114
200      A partir de 193.000
263    A partir de 2.775.543
271    A partir de 2.958.945
287      A partir de 959.482
295    A partir de 2.348.248
299    A partir de 1.060.000
322      A partir de 636.451
326    A partir de 1.763.862
330      A partir de 399.700
334    A partir de 2.845.874
346    A partir de 1.547.769
354      A partir de 197.255
374    A partir de 2.871.998
392    A partir de 1.181.161
Name: PRECO, dtype: object

In [34]:
df['VALOR1'] = (
    df['PRECO']
    .astype(str)
    .str.replace('.', '', regex=False)
    .str.replace(',', '.', regex=False)
)

In [35]:
# Convertendo tudo para numero
df['VALOR1'] = pd.to_numeric(df['VALOR1'], errors='coerce')

- Os valores que não forem númericos irão se tornar NaN

In [36]:
# Remoção dos NaN
df = df.dropna(subset=['VALOR1'])

In [37]:
# Conferindo se ainda resta algum NaN
assert df['VALOR1'].notna().all()

In [38]:
# Aepnas numeros reais
df['PRECO'].unique()

array(['1.400.000', '1.529.000', '1.395.000', ..., '152.000', '282.150',
       '1.447.000'], shape=(1866,), dtype=object)

In [39]:
df['VALOR1'].notna().all()

np.True_

In [40]:
# Conferindo de forma mais robusta
df.loc[
    df['PRECO'].astype(str).str.contains('sob|partir', case=False, na=False),
    'PRECO'
]

Series([], Name: PRECO, dtype: object)

In [41]:
df.dtypes

PRECO       object
AREA        object
QUARTOS     object
BAIRRO      object
VALOR      float64
VALOR1     float64
dtype: object

In [42]:
df.shape

(17654, 6)

In [43]:
df.drop(['PRECO','VALOR'], axis=1, inplace=True)

In [44]:
df.rename(columns={'VALOR1': 'VALOR'}, inplace= True)

In [45]:
df.head()

,AREA,QUARTOS,BAIRRO,VALOR
1,141 m²,2 Quartos,ASA NORTE,1400000.0
2,79 m²,2 Quartos,SUDOESTE,1529000.0
3,85 m²,2 Quartos,NOROESTE,1395000.0
5,58 m²,2 Quartos,AGUAS CLARAS,450000.0
6,186 m²,4 Quartos,SUDOESTE,3580000.0


#### 3.1 Remoção de registros com valores não determinísticos

Durante a análise da base, foi identificado que parte dos registros apresenta valores não determinísticos nas variáveis `PREÇO` e `AREA`, representados por expressões como _“Sob Consulta”_ e _“A partir de X”_, além de intervalos do tipo _“55 a 137 m²”_.

Esses registros não representam observações pontuais, mas sim intervalos ou valores indefinidos, geralmente associados a imóveis em lançamento ou estratégias comerciais. Como consequência, tais informações não refletem um valor real único e não podem ser interpretadas de forma precisa pelo modelo.

A imputação desses valores por média ou mediana poderia introduzir viés estatístico, além de adicionar ruído artificial ao conjunto de dados, comprometendo a capacidade de generalização do modelo.

Diante disso, optou-se pela remoção desses registros, priorizando a consistência e a qualidade estatística da base em detrimento da quantidade de observações. Considerando o tamanho do dataset, essa decisão não impacta de forma significativa a representatividade dos dados, mas contribui para um conjunto mais confiável para análise exploratória e modelagem preditiva.

#### 4. Tratamento e padronização da variável AREA

A variável `AREA` apresenta valores representados como texto, contendo a unidade de medida `(m²)` e, em alguns casos, formatações que impedem sua conversão direta para um tipo numérico. Essa estrutura inviabiliza a aplicação de estatísticas descritivas e o uso da variável em modelos de machine learning.

Dessa forma, tornou-se necessária a remoção da unidade de medida e a padronização dos valores, convertendo a coluna `AREA` para o formato numérico `(float)`. Essa transformação permite que a variável represente adequadamente a metragem dos imóveis, viabilizando análises exploratórias, visualizações e etapas posteriores de modelagem.

In [46]:
df['AREA_NUM'] = (
    df['AREA']
    .astype(str)
    .str.extract(r'(\d+)') # Extrai apenas os numeros
)

In [47]:
df['AREA_NUM'] = pd.to_numeric(df['AREA_NUM'], errors='coerce') # Se nao for numero vira NaN

In [48]:
df = df.dropna(subset=['AREA_NUM']) # Remove as linhas invalidas se existirem

In [49]:
df.loc[df['AREA'].str.contains('[a-zA-Z]', na=False), 'AREA'] # Existe apenas na original

1         141 m²
2          79 m²
3          85 m²
5          58 m²
6         186 m²
          ...   
17727     270 m²
17728     300 m²
17729     250 m²
17730    3,16 ha
17731      93 m²
Name: AREA, Length: 17649, dtype: object

In [50]:
assert df['AREA_NUM'].notna().all() # Numerico limpo

In [51]:
df['AREA_NUM'].describe() # Apenas fazendo um conferencia rapida

count    1.764900e+04
mean     1.049623e+04
std      1.141277e+06
min      0.000000e+00
25%      7.200000e+01
50%      1.800000e+02
75%      4.000000e+02
max      1.500000e+08
Name: AREA_NUM, dtype: float64

In [52]:
df.head()

,AREA,QUARTOS,BAIRRO,VALOR,AREA_NUM
1,141 m²,2 Quartos,ASA NORTE,1400000.0,141.0
2,79 m²,2 Quartos,SUDOESTE,1529000.0,79.0
3,85 m²,2 Quartos,NOROESTE,1395000.0,85.0
5,58 m²,2 Quartos,AGUAS CLARAS,450000.0,58.0
6,186 m²,4 Quartos,SUDOESTE,3580000.0,186.0


In [53]:
df = df.drop(columns=['AREA'])
df = df.rename(columns={'AREA_NUM': 'AREA'})

#### 5. Tratamento e padronização da variável QUARTOS

A coluna `QUARTOS` apresenta valores textuais que combinam números e descrições. Para viabilizar sua utilização analítica, os valores irão ser padronizados, mantendo apenas a quantidade numérica de quartos e convertendo a variável para formato numérico.

In [54]:
df['QUARTS_NUM'] = (
    df['QUARTOS']
    .astype(str)
    .str.extract(r'(\d+)')
)

In [55]:
df['QUARTS_NUM'] = pd.to_numeric(df['QUARTS_NUM'], errors='coerce')

In [56]:
df = df.dropna(subset=['QUARTS_NUM'])

In [57]:
df['QUARTS_NUM'].unique()

array([  2.,   4.,   3.,   6.,   1.,   5.,  20.,   7.,   9.,  17.,   8.,
        25.,  10.,  12.,  30.,  35.,  18.,  33.,  90.,  15.,  50.,  23.,
        13., 100.,  31.,  14.,  16.,  40.,  19.,  93.,  11.,  24.,  22.])

In [58]:
assert df['QUARTS_NUM'].notna().all()

In [59]:
df = df.drop(columns=['QUARTOS'])

In [60]:
df = df.rename(columns={'QUARTS_NUM': 'QUARTOS'})

In [61]:
df.head()

,BAIRRO,VALOR,AREA,QUARTOS
1,ASA NORTE,1400000.0,141.0,2.0
2,SUDOESTE,1529000.0,79.0,2.0
3,NOROESTE,1395000.0,85.0,2.0
5,AGUAS CLARAS,450000.0,58.0,2.0
6,SUDOESTE,3580000.0,186.0,4.0


In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15048 entries, 1 to 17731
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   BAIRRO   14741 non-null  object 
 1   VALOR    15048 non-null  float64
 2   AREA     15048 non-null  float64
 3   QUARTOS  15048 non-null  float64
dtypes: float64(3), object(1)
memory usage: 587.8+ KB


#### Encerramento

Nesta etapa, foram realizadas as principais ações de limpeza, padronização e validação do conjunto de dados, garantindo consistência e qualidade das variáveis utilizadas. Os dados foram preparados para permitir **análises estatísticas** confiáveis e uso adequado em etapas posteriores de exploração e modelagem.

Com o dataset devidamente tratado, o próximo passo consiste na **Análise Exploratória dos Dados (EDA)**, que será conduzida em um notebook separado, com foco na compreensão dos padrões, distribuições e relações entre as variáveis. Posteriormente, a etapa de modelagem preditiva será desenvolvida em um terceiro notebook, mantendo a organização e a clareza do fluxo do projeto.

In [63]:
# salvando o DataSet limpo

df.to_csv(r'C:\analise_imoveis\data\processed\imoveis_df_limpo.csv', index=False)